In [14]:
import pdfplumber
import pandas as pd
import os
import re
from datetime import datetime

In [22]:
pdf_folder = "C:\\Users\\dnava\\OneDrive\\Documentos\\Uni David\\Uni\\Planes semestrales\\Plan de estudios 5to semestre\\Tareas AM&IA\\TareasAprendizaje\\TareasAprendizaje\\practica1\\TicketsPDF"

# Verificar que la carpeta existe
if not os.path.exists(pdf_folder):
    print(f"Error: La carpeta '{pdf_folder}' no existe.")
    print("Por favor, ajusta la variable 'pdf_folder' con la ruta correcta.")
else:
    print(f"Directorio PDF: {pdf_folder}")
    archivos_pdf = [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
    print(f"Se encontraron {len(archivos_pdf)} archivos PDF")

Directorio PDF: C:\Users\dnava\OneDrive\Documentos\Uni David\Uni\Planes semestrales\Plan de estudios 5to semestre\Tareas AM&IA\TareasAprendizaje\TareasAprendizaje\practica1\TicketsPDF
Se encontraron 1000 archivos PDF


In [23]:
# Ejecuta esto para ver qué hay en la variable archivos_pdf
print("Contenido de archivos_pdf:")
print(archivos_pdf[:20])  # Muestra los primeros 20 elementos
print(f"Tipo de datos: {type(archivos_pdf)}")
print(f"Longitud: {len(archivos_pdf)}")

Contenido de archivos_pdf:
['Ticket_00B8DM.pdf', 'Ticket_026FPZ.pdf', 'Ticket_027GRW.pdf', 'Ticket_04IUR9.pdf', 'Ticket_05ICV4.pdf', 'Ticket_06WB1Z.pdf', 'Ticket_09PTL6.pdf', 'Ticket_0AV6UQ.pdf', 'Ticket_0BZHF6.pdf', 'Ticket_0C1I2C.pdf', 'Ticket_0CU5QI.pdf', 'Ticket_0EUVM2.pdf', 'Ticket_0FHWEB.pdf', 'Ticket_0FLXM2.pdf', 'Ticket_0IJIA3.pdf', 'Ticket_0IRYP2.pdf', 'Ticket_0JCB09.pdf', 'Ticket_0JPPWW.pdf', 'Ticket_0LXQMR.pdf', 'Ticket_0OGOGJ.pdf']
Tipo de datos: <class 'list'>
Longitud: 1000


In [ ]:
patrones = {
    'id_ticket': r'Ticket\s*#([A-Z0-9]+)',
    'fecha': r'Fecha:\s*(\d{2}/\d{2}/\d{4})',
    'metodo_pago': r'Método de pago:\s*([A-Za-z]+)[^\n]*',  # Solo captura la primera palabra
    'subtotal': r'Subtotal:\s*\$([\d,]+\.\d{2})',
    'descuento': r'Descuento:\s*\$([\d,]+\.\d{2})',
    'iva': r'IVA\s*\(.*\):\s*\$([\d,]+\.\d{2})',
    'total': r'Total:\s*\$([\d,]+\.\d{2})'  # Este debe capturar el valor correcto
}

def extraer_info_pdf(ruta_pdf):
    """Extrae información general de un ticket PDF"""
    info = {campo: None for campo in patrones.keys()}
    
    try:
        with pdfplumber.open(ruta_pdf) as pdf:
            texto_completo = ""
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                if texto:
                    texto_completo += texto + "\n"
            
            # Extraer información usando patrones regex mejorados
            # ID Ticket
            coincidencia_id = re.search(r'Ticket\s*#([A-Z0-9]+)', texto_completo)
            if coincidencia_id:
                info['id_ticket'] = coincidencia_id.group(1).strip()
            
            # Fecha
            coincidencia_fecha = re.search(r'Fecha:\s*(\d{2}/\d{2}/\d{4})', texto_completo)
            if coincidencia_fecha:
                info['fecha'] = coincidencia_fecha.group(1).strip()
            
            # Método de pago (MEJORADO)
            coincidencia_metodo = re.search(r'Método de pago:\s*([A-Za-z]+)', texto_completo)
            if coincidencia_metodo:
                info['metodo_pago'] = coincidencia_metodo.group(1).strip()
            
            # Subtotal
            coincidencia_subtotal = re.search(r'Subtotal:\s*\$([\d,]+\.\d{2})', texto_completo)
            if coincidencia_subtotal:
                info['subtotal'] = coincidencia_subtotal.group(1).replace(',', '')
            
            # Descuento
            coincidencia_descuento = re.search(r'Descuento:\s*\$([\d,]+\.\d{2})', texto_completo)
            if coincidencia_descuento:
                info['descuento'] = coincidencia_descuento.group(1).replace(',', '')
            else:
                info['descuento'] = '0.00'  # Valor por defecto si no hay descuento
            
            # IVA
            coincidencia_iva = re.search(r'IVA\s*\(.*\):\s*\$([\d,]+\.\d{2})', texto_completo)
            if coincidencia_iva:
                info['iva'] = coincidencia_iva.group(1).replace(',', '')
            
            # Total (MEJORADO - busca específicamente la línea Total)
            coincidencia_total = re.search(r'Total:\s*\$([\d,]+\.\d{2})', texto_completo)
            if coincidencia_total:
                info['total'] = coincidencia_total.group(1).replace(',', '')
            
            # Si no encontramos ID, usar el nombre del archivo
            if not info['id_ticket']:
                nombre_archivo = os.path.splitext(os.path.basename(ruta_pdf))[0]
                info['id_ticket'] = nombre_archivo.replace('Ticket_', '')
                
    except Exception as e:
        print(f"Error procesando {os.path.basename(ruta_pdf)}: {str(e)}")
        return None
    
    return info
    """Extrae información general de un ticket PDF"""
    info = {campo: None for campo in patrones.keys()}
    
    try:
        with pdfplumber.open(ruta_pdf) as pdf:
            texto_completo = ""
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                if texto:
                    texto_completo += texto + "\n"
            
            # Debug: Mostrar texto extraído para los primeros archivos
            nombre_archivo = os.path.basename(ruta_pdf)
            if nombre_archivo in ['Ticket_00B8DM.pdf', 'Ticket_026FPZ.pdf', 'Ticket_027GRW.pdf']:
                print(f"\n=== TEXTO EXTRAÍDO DE {nombre_archivo} ===")
                print(texto_completo[:500] + "..." if len(texto_completo) > 500 else texto_completo)
                print("=" * 50)
            
            # Extraer información usando patrones regex
            for campo, patron in patrones.items():
                coincidencia = re.search(patron, texto_completo, re.IGNORECASE)
                if coincidencia:
                    info[campo] = coincidencia.group(1).strip()
                else:
                    # Intentar con patrones alternativos si el principal falla
                    if campo == 'iva':
                        coincidencia_alt = re.search(r'IVA.*\$([\d,]+\.\d{2})', texto_completo, re.IGNORECASE)
                        if coincidencia_alt:
                            info[campo] = coincidencia_alt.group(1).strip()
            
            # Si no encontramos ID, usar el nombre del archivo
            if not info['id_ticket']:
                info['id_ticket'] = os.path.splitext(os.path.basename(ruta_pdf))[0].replace('Ticket_', '')
                
    except Exception as e:
        print(f"Error procesando {os.path.basename(ruta_pdf)}: {str(e)}")
        return None
    
    return info
    """Extrae información general de un ticket PDF"""
    info = {campo: None for campo in patrones.keys()}
    
    try:
        with pdfplumber.open(ruta_pdf) as pdf:
            texto_completo = ""
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                if texto:
                    texto_completo += texto + "\n"
            
            # Extraer información usando patrones regex
            for campo, patron in patrones.items():
                coincidencia = re.search(patron, texto_completo, re.IGNORECASE)
                if coincidencia:
                    info[campo] = coincidencia.group(1).strip()
            
            # Si no encontramos ID, usar el nombre del archivo
            if not info['id_ticket']:
                info['id_ticket'] = os.path.splitext(os.path.basename(ruta_pdf))[0]
                
    except Exception as e:
        print(f"Error procesando {os.path.basename(ruta_pdf)}: {str(e)}")
        return None
    
    return info

In [ ]:
def procesar_tickets(pdf_folder, max_archivos=None):
    """Procesa todos los archivos PDF en la carpeta y retorna un DataFrame"""
    
    # Obtener lista de archivos PDF
    archivos_pdf = [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
    
    if max_archivos:
        archivos_pdf = archivos_pdf[:max_archivos]
    
    print(f"Procesando {len(archivos_pdf)} archivos PDF...")
    
    # Crear DataFrame
    columnas = ['id_ticket', 'fecha', 'metodo_pago', 'subtotal', 'descuento', 'iva', 'total']
    tickets_df = pd.DataFrame(columns=columnas)
    
    # Procesar cada archivo
    for i, archivo in enumerate(archivos_pdf):
        ruta_completa = os.path.join(pdf_folder, archivo)
        info_ticket = extraer_info_pdf(ruta_completa)
        
        if info_ticket:
            tickets_df = pd.concat([tickets_df, pd.DataFrame([info_ticket])], ignore_index=True)
            print(f"✓ {archivo} - Extraído: {info_ticket['id_ticket']}")
        else:
            print(f"✗ {archivo} - Error en extracción")
        
        # Mostrar progreso
        if (i + 1) % 10 == 0:
            print(f"Procesados {i + 1}/{len(archivos_pdf)} archivos...")
    
    return tickets_df

In [ ]:
# Probar con los primeros 10 archivos
print("Procesando primeros 10 tickets...")
df_prueba = procesar_tickets(pdf_folder, max_archivos=10)

# Mostrar resultados
print("\nDataFrame resultante:")
display(df_prueba)

# Mostrar estadísticas
print("\nResumen de extracción:")
for columna in df_prueba.columns:
    no_nulos = df_prueba[columna].notna().sum()
    porcentaje = (no_nulos / len(df_prueba)) * 100
    print(f"{columna:12}: {no_nulos:2d}/{len(df_prueba)} ({porcentaje:5.1f}%)")

Procesando primeros 10 tickets...
Procesando 1000 archivos PDF...
✓ Ticket_00B8DM.pdf - Extraído: 00B8DM
✓ Ticket_026FPZ.pdf - Extraído: 026FPZ
✓ Ticket_027GRW.pdf - Extraído: 027GRW
✓ Ticket_04IUR9.pdf - Extraído: 04IUR9
✓ Ticket_05ICV4.pdf - Extraído: 05ICV4
✓ Ticket_06WB1Z.pdf - Extraído: 06WB1Z
✓ Ticket_09PTL6.pdf - Extraído: 09PTL6
✓ Ticket_0AV6UQ.pdf - Extraído: 0AV6UQ
✓ Ticket_0BZHF6.pdf - Extraído: 0BZHF6
✓ Ticket_0C1I2C.pdf - Extraído: 0C1I2C
Procesados 10/1000 archivos...
✓ Ticket_0CU5QI.pdf - Extraído: 0CU5QI
✓ Ticket_0EUVM2.pdf - Extraído: 0EUVM2
✓ Ticket_0FHWEB.pdf - Extraído: 0FHWEB
✓ Ticket_0FLXM2.pdf - Extraído: 0FLXM2
✓ Ticket_0IJIA3.pdf - Extraído: 0IJIA3
✓ Ticket_0IRYP2.pdf - Extraído: 0IRYP2
✓ Ticket_0JCB09.pdf - Extraído: 0JCB09
✓ Ticket_0JPPWW.pdf - Extraído: 0JPPWW
✓ Ticket_0LXQMR.pdf - Extraído: 0LXQMR
✓ Ticket_0OGOGJ.pdf - Extraído: 0OGOGJ
Procesados 20/1000 archivos...
✓ Ticket_0OHEIR.pdf - Extraído: 0OHEIR
✓ Ticket_0OU1FW.pdf - Extraído: 0OU1FW
✓ Ticket_0OUF9

,id_ticket,fecha,metodo_pago,subtotal,descuento,iva,total
0,00B8DM,31/07/2025,Transferencia,4814.12,0.00,770.26,5650.43
1,026FPZ,01/08/2025,Transferencia,6392.05,319.60,971.59,7068.63
2,027GRW,18/08/2025,Transferencia,6262.95,313.15,951.97,6852.83
3,04IUR9,27/06/2025,Efectivo,5591.15,0.00,894.58,6448.71
4,05ICV4,03/06/2025,Transferencia,5455.44,0.00,872.87,6203.88
...,...,...,...,...,...,...,...
995,ZXMVFK,16/07/2025,Tarjeta,29687.79,1484.39,4512.54,32864.70
996,ZXRK34,04/06/2025,Efectivo,4959.33,0.00,793.49,5739.97
997,ZZ2AUI,02/09/2025,Tarjeta,6290.11,314.51,956.10,6877.61
998,ZZ76LF,24/07/2025,Transferencia,6934.47,346.72,1054.04,7569.29



Resumen de extracción:
id_ticket   : 1000/1000 (100.0%)
fecha       : 1000/1000 (100.0%)
metodo_pago : 1000/1000 (100.0%)
subtotal    : 984/1000 ( 98.4%)
descuento   : 1000/1000 (100.0%)
iva         : 984/1000 ( 98.4%)
total       : 984/1000 ( 98.4%)


In [ ]:
#df_prueba.to_csv("tickets_info_gral.csv", index=False)

In [33]:
def extraer_productos_pdf(ruta_pdf):
    """Extrae información de productos de un ticket PDF usando patrones"""
    productos = []
    
    try:
        with pdfplumber.open(ruta_pdf) as pdf:
            texto_completo = ""
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                if texto:
                    texto_completo += texto + "\n"
            
            # Obtener ID del ticket
            id_ticket = None
            coincidencia_id = re.search(r'Ticket\s*#([A-Z0-9]+)', texto_completo)
            if coincidencia_id:
                id_ticket = coincidencia_id.group(1).strip()
            else:
                nombre_archivo = os.path.splitext(os.path.basename(ruta_pdf))[0]
                id_ticket = nombre_archivo.replace('Ticket_', '')
            
            # Patrón para encontrar productos: cantidad, producto, precio, total
            patron_producto = r'(?<!Cant Producto Precio Total\\n)(\d{1,4})\s+([a-zA-Z][^\n]+?)\s+\$([\d,]+\.\d{2,})\s+\$([\d,]+\.\d{2,})'
            
            coincidencias = re.findall(patron_producto, texto_completo)
            
            for cantidad, producto, precio, total in coincidencias:
                producto_info = {
                    'id_ticket': id_ticket,
                    'cantidad': int(cantidad),
                    'producto': producto.strip(),
                    'precio': float(precio.replace(',', '')),
                    'total_producto': float(total.replace(',', ''))
                }
                productos.append(producto_info)
                
    except Exception as e:
        print(f"Error procesando {os.path.basename(ruta_pdf)}: {str(e)}")
        return []
    
    return productos

In [34]:
def procesar_productos(pdf_folder, max_archivos=None):
    """Procesa productos de múltiples archivos PDF y retorna DataFrame"""
    
    archivos_pdf = [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
    
    if max_archivos:
        archivos_pdf = archivos_pdf[:max_archivos]
    
    print(f"Procesando productos de {len(archivos_pdf)} archivos PDF...")
    
    todos_productos = []
    
    for i, archivo in enumerate(archivos_pdf):
        ruta_completa = os.path.join(pdf_folder, archivo)
        productos = extraer_productos_pdf(ruta_completa)
        
        if productos:
            todos_productos.extend(productos)
            print(f"✓ {archivo} - {len(productos)} productos")
        else:
            print(f"✗ {archivo} - 0 productos")
        
        if (i + 1) % 5 == 0:
            print(f"Procesados {i + 1}/{len(archivos_pdf)} archivos...")
    
    productos_df = pd.DataFrame(todos_productos)
    return productos_df


In [35]:
print("Extracción de productos de los primeros 10 tickets...")
productos_df = procesar_productos(pdf_folder, max_archivos=1000)

# Mostrar resultados
print("\n" + "="*60)
print("DATAFRAME DE PRODUCTOS:")
print("="*60)
display(productos_df)

print(f"\nRESUMEN:")
print(f"Total de productos: {len(productos_df)}")
print(f"Tickets procesados: {productos_df['id_ticket'].nunique()}")
print(f"Productos por ticket (promedio): {len(productos_df) / productos_df['id_ticket'].nunique():.1f}")

# Mostrar algunos productos únicos
print(f"\nProductos únicos encontrados:")
print(productos_df['producto'].unique()[:15])

Extracción de productos de los primeros 10 tickets...
Procesando productos de 1000 archivos PDF...
✓ Ticket_00B8DM.pdf - 11 productos
✓ Ticket_026FPZ.pdf - 10 productos
✓ Ticket_027GRW.pdf - 11 productos
✓ Ticket_04IUR9.pdf - 13 productos
✓ Ticket_05ICV4.pdf - 10 productos
Procesados 5/1000 archivos...
✓ Ticket_06WB1Z.pdf - 12 productos
✓ Ticket_09PTL6.pdf - 10 productos
✓ Ticket_0AV6UQ.pdf - 11 productos
✓ Ticket_0BZHF6.pdf - 5 productos
✓ Ticket_0C1I2C.pdf - 13 productos
Procesados 10/1000 archivos...
✓ Ticket_0CU5QI.pdf - 10 productos
✓ Ticket_0EUVM2.pdf - 7 productos
✓ Ticket_0FHWEB.pdf - 13 productos
✓ Ticket_0FLXM2.pdf - 11 productos
✓ Ticket_0IJIA3.pdf - 12 productos
Procesados 15/1000 archivos...
✓ Ticket_0IRYP2.pdf - 14 productos
✓ Ticket_0JCB09.pdf - 12 productos
✓ Ticket_0JPPWW.pdf - 10 productos
✓ Ticket_0LXQMR.pdf - 10 productos
✓ Ticket_0OGOGJ.pdf - 15 productos
Procesados 20/1000 archivos...
✓ Ticket_0OHEIR.pdf - 10 productos
✓ Ticket_0OU1FW.pdf - 10 productos
✓ Ticket_0

,id_ticket,cantidad,producto,precio,total_producto
0,00B8DM,3,Pan Baguette,31.20,93.59
1,00B8DM,1,Pollo 1kg,123.24,123.24
2,00B8DM,1,Sartén antiadherente,352.00,352.00
3,00B8DM,8,Huevos 12pz,79.19,633.48
4,00B8DM,1,Detergente 2kg,88.05,88.05
...,...,...,...,...,...
11343,ZZWCTH,3,Detergente 2kg,91.86,275.59
11344,ZZWCTH,2,Sartén antiadherente,332.88,665.75
11345,ZZWCTH,1,Huevos 12pz,76.18,76.18
11346,ZZWCTH,2,Sartén antiadherente,364.92,729.85



RESUMEN:
Total de productos: 11348
Tickets procesados: 1000
Productos por ticket (promedio): 11.3

Productos únicos encontrados:
['Pan Baguette' 'Pollo 1kg' 'Sartén antiadherente' 'Huevos 12pz'
 'Detergente 2kg' 'Leche 1L' 'Audífonos' 'Licuadora' 'Refresco 2L'
 'Microondas' 'Arroz 1kg' 'Celular básico']


In [36]:
productos_df.to_csv("tickets_productos.csv", index=False)